## HW 5.0
What is a data warehouse? What is a Star schema? When is it used?

## HW 5.1
In the database world What is 3NF? Does machine learning use data in 3NF? If so why?   
In what form does ML consume data?  
Why would one use log files that are denormalized?  


## HW 5.2

Using MRJob, implement a hashside join (memory-backed map-side) for left, right and inner joins. 

Run your code on the data used in HW 4.4: (Recall HW 4.4: Find the most frequent visitor of each page using mrjob and the output of 4.2  (i.e., transfromed log file)). 

In this output please include the webpage URL, webpageID and Visitor ID.:

Justify which table you chose as the Left table in this hashside join.

Please report the number of rows resulting from:

1. Left joining Table Left with Table Right
2. Right joining Table Left with Table Right
3. Inner joining Table Left with Table Right



In [9]:
%load_ext autoreload
%autoreload 2

In [24]:
%%writefile MRJoin_5_2.py

from mrjob.job import MRJob, MRStep
import mrjob
import csv
import sys

class MRJoin(MRJob):
    urls = {} # key = pageId, value = url
    keys_emitted = set() # Set of keys of all emitted urls. Used for left join.
    
    def configure_options(self):
        super(MRJoin, self).configure_options()
        self.add_passthrough_option(
            '--joinType', type='str', default="inner")
        
    def load_options(self, args):
        super(MRJoin, self).load_options(args)
        self.joinType = self.options.joinType
        
    def mapper_init(self):
        # Load URL info data file into memory.  
        # Line format: 
        # 1287,/autoroute
        with open("processed_urls.data", "r") as f:
            for fields in csv.reader(f):
                self.urls[fields[0]] = fields[1]

    def mapper(self, line_no, line):
        # Line format: 
        # V,1000,1,C,10001
        fields = csv.reader([line]).next()
        
        key = fields[1]
        url = None
        toEmit = False
        
        if key in self.urls:
            url = self.urls[key]
            
        if self.joinType == "right":
            toEmit = True
        elif self.joinType == "left":
            if url is not None:
                toEmit = True
                self.keys_emitted.add(key) # Remember what we have emitted
        else: # inner join
            if url is not None:
                toEmit = True
        
        if toEmit:
            # Output format
            # pageid, url,V,1,C,10001
            yield key, (url, fields[0], fields[2], fields[3], fields[4])
        
    def mapper_final(self):
        if self.joinType == "left":
            # Emit all the remaining urls
            remaining = set(self.urls.keys()) - self.keys_emitted
            for key in remaining:
                yield key, (self.urls[key], None, None, None, None)
                                
    def steps(self):
        return [
            MRStep(mapper_init=self.mapper_init,
                   mapper=self.mapper,
                   mapper_final=self.mapper_final)
            ]
    
if __name__ == '__main__':
    MRJoin.run()


Overwriting MRJoin_5_2.py


In [27]:
from MRJoin_5_2 import MRJoin

for joinType in ["left", "right", "inner"]:
    mr_job = MRJoin(args=['processed_anonymous-msweb.data', 
                        '--file', 'processed_urls.data', # broadcast to every mapper
                        "--strict-protocols",
                        '--joinType', joinType])

    with mr_job.make_runner() as runner: 
        runner.run()

        lines = []
        for line in runner.stream_output():
            lines.append(line)
            
        print "Join type:" + joinType
        print "Number of records:" + str(len(lines))
        print "First 5 lines:"
        for i in range(5):
            print lines[i].strip()
            
        print


Join type:left
Number of records:98663
First 5 lines:
"1000"	["/regwiz", "V", "1", "C", "10001"]
"1001"	["/support", "V", "1", "C", "10001"]
"1002"	["/athome", "V", "1", "C", "10001"]
"1001"	["/support", "V", "1", "C", "10002"]
"1003"	["/kb", "V", "1", "C", "10002"]

Join type:right
Number of records:98654
First 5 lines:
"1000"	["/regwiz", "V", "1", "C", "10001"]
"1001"	["/support", "V", "1", "C", "10001"]
"1002"	["/athome", "V", "1", "C", "10001"]
"1001"	["/support", "V", "1", "C", "10002"]
"1003"	["/kb", "V", "1", "C", "10002"]

Join type:inner
Number of records:98654
First 5 lines:
"1000"	["/regwiz", "V", "1", "C", "10001"]
"1001"	["/support", "V", "1", "C", "10001"]
"1002"	["/athome", "V", "1", "C", "10001"]
"1001"	["/support", "V", "1", "C", "10002"]
"1003"	["/kb", "V", "1", "C", "10002"]

